In [ ]:
import pandas as pd
import os
import subprocess
encoding="utf-8"

def getBlocks(txtFile):
    with open(txtFile,"r") as f:
        textString=f.read()
    blocks=textString.split("\n")
    return blocks

def ckGetAll(gitProject,saveFolder):
    CKAnalyze=["java","-jar","ck-0.7.1-SNAPSHOT-jar-with-dependencies.jar",gitProject,"False","0","True",saveFolder]
    subprocess.run(CKAnalyze,cwd=os.path.join(os.getcwd(),"ck_anonymous\\ck\\target"),check=True)
    
def ckGetOrigin(gitProject,saveFolder):
    ckFolder="C:\\Users\\sugii syuji\\ck_line\\ck\\target"
    CKAnalyze=["java","-jar","ck-0.7.1-SNAPSHOT-jar-with-dependencies.jar",gitProject,"False","0","True",saveFolder]
    subprocess.run(CKAnalyze,cwd=ckFolder,check=True)
    
def joinClassFile(allFile,originFile,saveFile):
    all=pd.read_csv(allFile)
    origin=pd.read_csv(originFile)
    origin=origin[["file","line"]]
    #origin=origin.drop_duplicates()
    join=all.merge(origin,how="inner",on=["file","line"])
    join.to_csv(saveFile,index=False,encoding=encoding)
    
def joinMethodFile(allFile,originFile,saveFile):
    all=pd.read_csv(allFile)
    origin=pd.read_csv(originFile)
    origin=origin[["file","line"]]
    join=all.merge(origin,how="inner",on=["file","line"])
    join.to_csv(saveFile,index=False,encoding=encoding)
    
def gitCheckout(gitProject,version):
    checkout=["git","checkout",version]
    subprocess.run(checkout,cwd=gitProject,check=True)
    
def doAll(gitProject,saveFolder,saveFileName):
    allFile=os.path.join(saveFolder,saveFileName+"all")
    originFile=os.path.join(saveFolder,saveFileName+"origin")
    ckGetAll(gitProject,allFile)
    ckGetOrigin(gitProject,originFile)
    allClassFile=allFile+"class.csv"
    originClassFile=originFile+"class.csv"
    allMethodFile=allFile+"method.csv"
    originMethodFile=originFile+"method.csv"
    saveClassFile=os.path.join(saveFolder,saveFileName+"class.csv")
    saveMethodFile=os.path.join(saveFolder,saveFileName+"method.csv")
    joinClassFile(allClassFile,originClassFile,saveClassFile)
    joinMethodFile(allMethodFile,originMethodFile,saveMethodFile)
    
def modifyCKFile(saveFolder,saveFileName):
    allFile=os.path.join(saveFolder,saveFileName+"all")
    originFile=os.path.join(saveFolder,saveFileName+"origin")
    allClassFile=allFile+"class.csv"
    originClassFile=originFile+"class.csv"
    allMethodFile=allFile+"method.csv"
    originMethodFile=originFile+"method.csv"
    saveClassFile=os.path.join(saveFolder,saveFileName+"class.csv")
    saveMethodFile=os.path.join(saveFolder,saveFileName+"method.csv")
    joinClassFile(allClassFile,originClassFile,saveClassFile)
    joinMethodFile(allMethodFile,originMethodFile,saveMethodFile)
    
def editClass(Spoonfile, CKfile, outPutFile):   
    df = pd.read_csv(CKfile,encoding=encoding,encoding_errors="replace")
    df = df[["file","class","line","type", "dit", "fanin", "fanout","lcc","lcom*","loc","noc","rfc","innerClassesQty","totalFieldsQty","totalMethodsQty","wmc"]]
    CK=df.copy()
    CK=CK.drop_duplicates()
    CK["file"] = CK["file"].str.replace("sugii syuji","syuuj")
    CK["class"] = CK["class"].str.replace("$Anonymous", "$")
    spoon = pd.read_csv(Spoonfile,encoding=encoding,encoding_errors="replace")
    spoon=spoon.drop_duplicates()
    join = CK.merge(spoon, how="inner", on=["file","line"])
    #join["class"]=join["class"].str.replace("$",".",)
    join.to_csv(
        outPutFile,mode="w",index=False,encoding=encoding
    )

def editMethod(SpoonFile, CKFile, outPutFile):
    df = pd.read_csv(CKFile,encoding=encoding ,encoding_errors="replace")
    df = df[["file", "class", "method","fanin","fanout", "wmc", "loc","returnsQty","variablesQty","parametersQty","loopQty","anonymousClassesQty","innerClassesQty","line"]]
    CK=df.copy()
    CK=CK.drop_duplicates()
    CK["file"]=CK["file"].str.replace("sugii syuji","syuuj")
    CK["class"] = CK["class"].str.replace("$Anonymous", "$")    
    CK["method"]=CK["method"].str.replace(r"<[^>]*>", "", regex=True)

    spoon = pd.read_csv(SpoonFile,encoding=encoding,encoding_errors="replace")
    spoon=spoon.drop_duplicates()
    spoon["method"]=spoon["method"].str.replace("$",".")
    spoon["method"]=spoon["method"].str.replace(" ","")

    join = CK.merge(spoon, how="inner", on=["file", "line"])
    #join["class"]=join["class"].str.replace("$",".")
    join.to_csv(
        outPutFile,mode="w",index=False,encoding=encoding
    )
    
def joinCKSPOONFile(ckFolder,spoonFolder,saveFolder,saveFileName):
    ckClass=saveFileName+"originclass.csv"
    spoonClass=saveFileName+"spoonClass.csv"
    ckMethod=saveFileName+"originmethod.csv"
    spoonMethod=saveFileName+"spoonMethod.csv"
    
    ckClassFile=os.path.join(ckFolder,ckClass)
    spoonClassFile=os.path.join(spoonFolder,spoonClass)
    ckMethodFile=os.path.join(ckFolder,ckMethod)
    spoonMethodFie=os.path.join(spoonFolder,spoonMethod)
    
    outPutClassFile=os.path.join(saveFolder,saveFileName+"joinClass.csv")
    outPutMethodFile=os.path.join(saveFolder,saveFileName+"joinMethod.csv")
    editClass(spoonClassFile,ckClassFile,outPutClassFile)
    editMethod(spoonMethodFie,ckMethodFile,outPutMethodFile)

In [ ]:
# オリジナル と 重複OK をCKで計測した後，file名とlineで結合する
import shutil
projectPath="C:\\Users\\sugii syuji\\gitProject"
blocks=getBlocks("version2.txt")

for block in blocks:
    projectInfo=block.split(",")
    projectName=projectInfo[0]
    versions=[projectInfo[1],projectInfo[2]]
    gitProject=os.path.join(projectPath,projectName)
    saveFolder=os.path.join(os.getcwd(),"data5",projectName)
    if os.path.isdir(saveFolder):
        shutil.rmtree(saveFolder)
    os.makedirs(saveFolder)
    i=1
    for version in versions:
        gitCheckout(gitProject,version)
        doAll(gitProject,saveFolder,str(i))
        i=i+1

In [ ]:
#ck同士　の結合データの表示形式を変更する
#joinClassFile joinMethodFileを編集する必要あり
blocks=getBlocks("version2.txt")
for block in blocks:
    projectInfo=block.split(",")
    projectName=projectInfo[0]
    saveFolder=os.path.join(os.getcwd(),"data5",projectName)
    i=1
    while i<=2:
        modifyCKFile(saveFolder,str(i))
        i=i+1

In [ ]:
# オリジナル と 結合後 のクラス・メソッド数をprintする
blocks=getBlocks("version2.txt")
for block in blocks:
    projectInfo=block.split(",")
    projectName=projectInfo[0]
    projectFolder=os.path.join(os.getcwd(),"data4",projectName)
    df1=pd.read_csv(os.path.join(projectFolder,"1class.csv"))
    df2=pd.read_csv(os.path.join(projectFolder,"1originclass.csv"))
    df3=pd.read_csv(os.path.join(projectFolder,"1method.csv"))
    df4=pd.read_csv(os.path.join(projectFolder,"1originmethod.csv"))
    print(projectName)
    print("class : ",len(df1),len(df2))
    print("method : ",len(df3),len(df4))
    print()

In [18]:
# 上で結合したもの と spoonで計測したもの を結合する
import shutil
blocks=getBlocks("version.txt")
for block in blocks:
    projectInfo=block.split(",")
    projectName=projectInfo[0]
    saveFolder=os.path.join(os.getcwd(),"joinCKSPOON_4",projectName)
    if os.path.isdir(saveFolder):
        shutil.rmtree(saveFolder)
    os.makedirs(saveFolder)
    ckFolder=os.path.join(os.getcwd(),"data5",projectName)
    spoonFolder=os.path.join(os.getcwd(),"data1",projectName)
    i=1
    while i<=2:
        joinCKSPOONFile(ckFolder,spoonFolder,saveFolder,str(i))
        i=i+1
    

In [ ]:
#結合後と結合前のクラス及びメソッド数
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("display.max_colwidth",None)
blocks=getBlocks("version.txt")
for block in blocks:
    projectInfo=block.split(",")
    projectName=projectInfo[0]
    joinFolder=os.path.join(os.getcwd(),"joinCKSPOON_4",projectName)
    ckFolder=os.path.join(os.getcwd(),"data5",projectName)
    joinClass=pd.read_csv(os.path.join(joinFolder,"2joinClass.csv"),encoding=encoding)
    joinMethod=pd.read_csv(os.path.join(joinFolder,"2joinMethod.csv"))
    ckClass=pd.read_csv(os.path.join(ckFolder,"2originclass.csv"))
    ckMethod=pd.read_csv(os.path.join(ckFolder,"2originmethod.csv"))
    print(projectName)
    tmpClass=joinClass[["file","class_x"]]
    tmpMethod=joinMethod[["file","class_x","method_x"]]
    print("class : ",len(joinClass),len(ckClass))
    print("method : ",len(joinMethod),len(ckMethod))
    display(tmpClass[tmpClass.duplicated()])
    display(tmpMethod[tmpMethod.duplicated()])

    print()

In [ ]:
#重複メソッドの出力
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("display.max_colwidth",None)
blocks=getBlocks("version.txt")
for block in blocks:
    projectInfo=block.split(",")
    projectName=projectInfo[0]
    joinFolder=os.path.join(os.getcwd(),"joinCKSPOON_3",projectName)
    ckFolder=os.path.join(os.getcwd(),"data5",projectName)
    joinClass=pd.read_csv(os.path.join(joinFolder,"1joinClass.csv"))
    joinMethod=pd.read_csv(os.path.join(joinFolder,"1joinMethod.csv"))
    print(projectName)
    #joinClass_tmp=joinClass[["file","class","type"]]
    joinMethod_tmp=joinMethod[["file","class","line"]]
    #display("重複class : ",joinClass_tmp[joinClass_tmp.duplicated()])
    display("重複メソッド : ",joinMethod_tmp[joinMethod_tmp.duplicated()])
    print()

In [ ]:
#それぞれの type 数
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("display.max_colwidth",None)
types=["class","enum","anonymous","innerclass","interface"]
joinFolder="joinCKSPOON_3"
originFolder="data5"

blocks=getBlocks("version.txt")
for block in blocks:
    info=block.split(",")
    projectName=info[0]
    joinClass=pd.read_csv(os.path.join(joinFolder,projectName,"1joinClass.csv"))
    originClass=pd.read_csv(os.path.join(originFolder,projectName,"1originclass.csv"))
    print(projectName)
    for type in types:
        print(type," : ",len(joinClass[joinClass["type"]==type]),len(originClass[originClass["type"]==type]))
    print()

In [19]:
#全ての結合ファイルを1つに
joinFolder="joinCKSPOON_4"

df=pd.DataFrame()
blocks=getBlocks("version.txt")
for block in blocks:
    info=block.split(",")
    projectName=info[0]
    joinClass=pd.read_csv(os.path.join(joinFolder,projectName,"1joinClass.csv"),encoding_errors="replace")
    df=pd.concat([df,joinClass])
    
df.to_csv("connectClass.csv")